<a href="https://colab.research.google.com/github/cutefishaep/BRender/blob/main/BRender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 BRender v1.5

**BRender** is a Google Colab notebook designed to render Blender projects using cloud GPU resources. It accepts direct file uploads or Google Drive links and supports both animation and single-frame rendering.

## Features

* **Version Control:** Supports user-defined Blender versions (tested from 2.79b up to 4.5.x).
* **Input Flexibility:** Accepts single `.blend` files or `.zip` archives (for projects with external assets/textures).
* **Google Drive Integration:** Direct download from Google Drive links to bypass local upload limits.
* **Interactive Selection:** Automatically scans extracted archives and prompts the user to select the specific `.blend` file to render.
* **Rendering Pipelines:** Support for CUDA and OptiX render devices with Hybrid (CPU+GPU) fallback.
* **Output Management:** Automatically groups rendered frames into a ZIP archive for a single download.

## How to Use

1.  **Configure Parameters**
    Adjust the settings in the **Configuration Parameters** cell before running:
    * `blender_version`: Select or type the specific version needed.
    * `upload_method`: Choose between **Upload** (local file) or **Google Drive Link**.
    * `renderer_type`: Select **CUDA** (standard) or **OptiX** (RTX acceleration).
    * `animation`: Check to render a range; uncheck for a single frame.

2.  **Run the Script**
    Execute the cell. The script will install the necessary dependencies (including `libtcmalloc`) and the selected Blender binary.

3.  **Provide Input File**
    * If **Upload** was selected: Use the "Choose Files" button when prompted.
    * If **Google Drive Link** was selected: Ensure the link has "Anyone with the link" access or is a direct download ID.

4.  **Select Scene**
    If multiple `.blend` files are detected in a ZIP archive, enter the corresponding number into the input field to confirm which file to render.

5.  **Download**
    Upon completion, the rendered outputs are zipped (if `zip_files` is enabled) and downloaded automatically.

## Configuration Details

| Parameter | Option/Type | Description |
| :--- | :--- | :--- |
| `upload_method` | Upload / Drive | Method for importing the project file. |
| `renderer_type` | CUDA / OptiX | Render device API. *Note: OptiX requires supported hardware.* |
| `gpu_enabled` | Boolean | Toggles GPU visibility to Cycles. |
| `cpu_enabled` | Boolean | Adds CPU to the device pool (Hybrid rendering). |
| `output_name` | String | Output filename format (use `#` for frame padding, e.g., `render_####`). |
| `zip_files` | Boolean | Compresses all output frames into one archive before downloading. |

## Credits
Developed and maintained by **@cutefishrbx** on TikTok.

In [ ]:
# @title **Configuration Parameters**
import os
import shutil
import requests
from google.colab import files
from IPython.display import clear_output

blender_version = '4.5.5' #@param ['2.79b','2.83.20','2.93.18','3.3.21','3.6.23','4.2.14','4.5.5'] {allow-input:true}
upload_method = "Upload" #@param ["Upload","Google Drive Link"]
gdrive_link = "" #@param {type:"string"}
animation = True #@param {type:"boolean"}
start_frame = 0 #@param {type:"integer"}
end_frame = 67 #@param {type:"integer"}
output_name = 'blender-##' #@param {type:"string"}
zip_files = True #@param {type:"boolean"}
renderer_type = "CUDA" #@param ["CUDA","OptiX"]
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}
debug = False #@param {type:"boolean"}

def print_debug(message):
    if debug:
        print(message)

def clear_if_not_debug():
    if not debug:
        clear_output(wait=True)

def is_libtcmalloc_installed():
    try:
        result = !dpkg -l | grep libtcmalloc-minimal4
        return len(result) > 0
    except:
        return False

def is_blender_installed(version):
    return os.path.exists(f"/content/{version}/blender")

def detect_file_type(file_path):
    try:
        with open(file_path,'rb') as f:
            header=f.read(8)
        if header.startswith(b'PK'): return 'zip'
        if header.startswith(b'BLENDER'): return 'blend'
        return 'unknown'
    except:
        return 'unknown'

def find_blend_files(directory):
    out=[]
    for r,_,fs in os.walk(directory):
        for f in fs:
            if f.lower().endswith('.blend'):
                out.append(os.path.relpath(os.path.join(r,f),directory))
    return out

def get_blender_download_url(version):
    urls={
        '2.79b':"https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
        '2.83.20':"https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz"
    }
    if version in urls: return urls[version]
    major=".".join(version.split('.')[:2])
    return f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{major}/blender-{version}-linux-x64.tar.xz"

def create_gpu_config():
    config = f"""import re,bpy
scene=bpy.context.scene
scene.cycles.device='GPU'
prefs=bpy.context.preferences
prefs.addons['cycles'].preferences.get_devices()
cprefs=prefs.addons['cycles'].preferences
for t in ('CUDA','OPENCL','NONE'):
    try:
        cprefs.compute_device_type=t
        break
    except:
        pass
for d in cprefs.devices:
    d.use = {gpu_enabled} if not re.match('intel',d.name,re.I) else {cpu_enabled}
"""
    with open('setgpu.py','w') as f:
        f.write(config)

def create_transparent_config():
    script = """import bpy
scene=bpy.context.scene
scene.render.film_transparent=True
scene.render.image_settings.file_format='PNG'
scene.render.image_settings.color_mode='RGBA'
scene.render.image_settings.color_depth='8'
"""
    with open('set_transparent.py','w') as f:
        f.write(script)

def main():
    global renderer_type
    os.chdir('/content')

    gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
    print("GPU:", gpu_info[0])

    if renderer_type=="OptiX":
        unsupported=["Tesla K80","Tesla P4","Tesla P100","Tesla V100"]
        if any(u in gpu_info[0] for u in unsupported):
            print("⚠️ OptiX not supported, switching to CUDA")
            renderer_type="CUDA"

    print("Renderer:", renderer_type)

    clear_if_not_debug()
    print("\n--- Dependencies ---")
    if not is_libtcmalloc_installed():
        os.environ["LD_PRELOAD"] = ""
        !apt update && apt install libtcmalloc-minimal4 -y
    os.environ["LD_PRELOAD"]="/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"

    clear_if_not_debug()
    print("\n--- File Processing ---")

    input_filename=""

    # UPLOAD langsung tanpa rename atau detect
    if upload_method == "Upload":
        uploaded = files.upload()
        raw = list(uploaded.keys())[0]
        input_filename = raw
        shutil.move(raw, input_filename)

    # Google Drive pakai detect + rename
    elif upload_method == "Google Drive Link" and gdrive_link:
        import gdown
        file_id=""
        if 'drive.google.com/file/d/' in gdrive_link:
            file_id=gdrive_link.split('drive.google.com/file/d/')[1].split('/')[0]
        elif 'id=' in gdrive_link:
            file_id=gdrive_link.split('id=')[1].split('&')[0]
        else:
            file_id=gdrive_link

        gdown.download(f"https://drive.google.com/uc?id={file_id}", "temp_file")
        ftype = detect_file_type("temp_file")

        if ftype=="zip": input_filename="input.zip"
        elif ftype=="blend": input_filename="input.blend"
        else: input_filename="input.unknown"

        os.rename("temp_file", input_filename)

    else:
        raise SystemExit("❌ No file provided")

    clear_if_not_debug()
    print("\n--- Processing ---")

    !rm -rf render && mkdir render

    # Upload langsung: cukup cek extension, tidak deteksi header
    if upload_method=="Upload":
        if input_filename.lower().endswith(".zip"):
            !unzip -o $input_filename -d render/
        elif input_filename.lower().endswith(".blend"):
            shutil.copy(input_filename, "render/")
        else:
            raise SystemExit("❌ Upload harus .blend atau .zip")

    # Google Drive masih pakai detect
    else:
        ftype = detect_file_type(input_filename)
        if ftype=="zip":
            !unzip -o $input_filename -d render/
        elif ftype=="blend":
            shutil.copy(input_filename,"render/")
        else:
            raise SystemExit("❌ Unsupported file format")

    blend_files = find_blend_files("/content/render")
    if not blend_files:
        raise SystemExit("❌ .blend not found")

    blend_file_path = blend_files[0]

    clear_if_not_debug()
    print("\n--- Blender Setup ---")

    if not is_blender_installed(blender_version):
        url=get_blender_download_url(blender_version)
        base=os.path.basename(url)
        !mkdir -p $blender_version
        !wget -nc $url -O $base
        !tar -xkf $base -C ./$blender_version --strip-components=1
    else:
        print_debug("Blender already installed")

    clear_if_not_debug()
    print("\n--- GPU Config ---")
    create_gpu_config()
    create_transparent_config()

    clear_if_not_debug()
    print("\n--- Rendering ---")

    !rm -rf output && mkdir output
    output_path="/content/output/"+output_name
    os.chdir(f"/content/{blender_version}")

    renderer = renderer_type.upper()

    if animation:
        if start_frame==end_frame:
            !./blender -b "/content/render/{blend_file_path}" -P "/content/setgpu.py" -E CYCLES -o "{output_path}" -noaudio -a -- --cycles-device "{renderer}"
        else:
            !./blender -b "/content/render/{blend_file_path}" -P "/content/setgpu.py" -E CYCLES -o "{output_path}" -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
    else:
        !./blender -b "/content/render/{blend_file_path}" -P "/content/setgpu.py" -P "/content/set_transparent.py" -E CYCLES -o "{output_path}" -noaudio -f $start_frame -- --cycles-device "{renderer}"

    clear_if_not_debug()
    print("\n--- Download ---")

    os.chdir("/content")
    _,_,fs = next(os.walk("output"))
    if not fs:
        raise SystemExit("❌ No output rendered")

    if len(fs)==1:
        files.download(f"output/{fs[0]}")
    else:
        if zip_files:
            zipname = output_name.replace("#","")+"render"
            shutil.make_archive(zipname,"zip","output")
            files.download(zipname+".zip")
        else:
            for f in fs:
                files.download(f"output/{f}")

    print("✓ Done")

if __name__=="__main__":
    main()
